In [17]:
import os
from skimage import io
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import shutil
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight
import seaborn as sns

In [19]:
batch_size = 32
img_height = 300
img_width = 200


In [ ]:
classes_weights = tf.constant([0.1, 1.0])
cross_entropy = tf.nn.weighted_cross_entropy_with_logits(logits=logits, targets=labels, pos_weight=classes_weights)

In [3]:
num_classes = 3
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(64, 7, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

In [4]:
def get_model_name():
    import time
    run=time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return run

In [5]:
recall=tf.keras.metrics.Recall(class_id=None)

In [6]:
root_logdir=os.path.join(os.curdir, 'my_logs')

In [7]:
def get_run_logdir():
    import time
    run_id=time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [8]:
run_logdir=get_run_logdir()
model_name=get_model_name()

In [15]:
callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-5 less"
        min_delta=1e-5,
        # "no longer improving" being further defined as "for at least 30 epochs"
        patience=45,
        verbose=1,), 
    keras.callbacks.TensorBoard(run_logdir,
                              histogram_freq=2,
                              write_graph=True,
                              write_images=True),
    keras.callbacks.ModelCheckpoint(filepath='models/'+model_name ,save_best_only=True)]

In [20]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=30, width_shift_range=0.3,
    height_shift_range=0.3, fill_mode='nearest', cval=0.0,
    horizontal_flip=True, vertical_flip=False, zoom_range=0.25, brightness_range=(0.6, 1.4))
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator()

In [21]:
train_generator = train_datagen.flow_from_directory(
        'plants/train',
        target_size=(300, 200),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'plants/val',
        target_size=(300, 200),
        batch_size=32,
        class_mode='categorical')

Found 1968 images belonging to 3 classes.
Found 422 images belonging to 3 classes.


In [23]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[recall])

In [ ]:
epochs=400
history = model.fit(
  train_generator,
  validation_data=validation_generator,
  epochs=epochs,
    callbacks=[callbacks])

Epoch 1/400
 1/62 [..............................] - ETA: 0s - loss: 1.1037 - recall: 0.0000e+00WARNING:tensorflow:From /Users/johnwilliams/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
62/62 [==============================] - ETA: 0s - loss: 0.8582 - recall: 0.6286WARNING:tensorflow:From /Users/johnwilliams/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets writt

62/62 [==============================] - 166s 3s/step - loss: 0.7095 - recall: 0.6723 - val_loss: 0.6955 - val_recall: 0.6706
Epoch 37/400
62/62 [==============================] - 166s 3s/step - loss: 0.7051 - recall: 0.6570 - val_loss: 0.7272 - val_recall: 0.5427
Epoch 38/400
62/62 [==============================] - 166s 3s/step - loss: 0.7038 - recall: 0.6621 - val_loss: 0.7135 - val_recall: 0.6754
Epoch 39/400
62/62 [==============================] - 166s 3s/step - loss: 0.7022 - recall: 0.6728 - val_loss: 0.6998 - val_recall: 0.6066
Epoch 40/400
62/62 [==============================] - 166s 3s/step - loss: 0.7002 - recall: 0.6657 - val_loss: 0.7019 - val_recall: 0.6943
Epoch 41/400
62/62 [==============================] - 166s 3s/step - loss: 0.7095 - recall: 0.6768 - val_loss: 0.7035 - val_recall: 0.6090
Epoch 42/400
62/62 [==============================] - 188s 3s/step - loss: 0.6917 - recall: 0.6606 - val_loss: 0.7143 - val_recall: 0.6185
Epoch 43/400
62/62 [====================

62/62 [==============================] - 232s 4s/step - loss: 0.6516 - recall: 0.7093 - val_loss: 0.7045 - val_recall: 0.6445
Epoch 89/400
62/62 [==============================] - 241s 4s/step - loss: 0.6519 - recall: 0.6972 - val_loss: 0.6820 - val_recall: 0.6730
Epoch 90/400
62/62 [==============================] - 243s 4s/step - loss: 0.6267 - recall: 0.7200 - val_loss: 0.6677 - val_recall: 0.6564
Epoch 91/400
62/62 [==============================] - 247s 4s/step - loss: 0.6142 - recall: 0.7149 - val_loss: 0.6334 - val_recall: 0.7156
Epoch 92/400
62/62 [==============================] - 248s 4s/step - loss: 0.6170 - recall: 0.7236 - val_loss: 0.6286 - val_recall: 0.7370
Epoch 93/400
62/62 [==============================] - 243s 4s/step - loss: 0.6220 - recall: 0.7149 - val_loss: 0.6471 - val_recall: 0.6777
Epoch 94/400
62/62 [==============================] - 235s 4s/step - loss: 0.6261 - recall: 0.7170 - val_loss: 0.6300 - val_recall: 0.7133
Epoch 95/400
62/62 [====================

Epoch 137/400
62/62 [==============================] - 235s 4s/step - loss: 0.5119 - recall: 0.7749 - val_loss: 0.6258 - val_recall: 0.7275
Epoch 138/400
62/62 [==============================] - 240s 4s/step - loss: 0.5367 - recall: 0.7510 - val_loss: 0.6100 - val_recall: 0.7299
Epoch 139/400
62/62 [==============================] - 243s 4s/step - loss: 0.5264 - recall: 0.7617 - val_loss: 0.6382 - val_recall: 0.7393
Epoch 140/400
62/62 [==============================] - 245s 4s/step - loss: 0.5215 - recall: 0.7536 - val_loss: 0.6418 - val_recall: 0.7156
Epoch 141/400
62/62 [==============================] - 243s 4s/step - loss: 0.5203 - recall: 0.7591 - val_loss: 0.6047 - val_recall: 0.6991
Epoch 142/400
62/62 [==============================] - 246s 4s/step - loss: 0.5118 - recall: 0.7622 - val_loss: 0.5939 - val_recall: 0.7346
Epoch 143/400
62/62 [==============================] - 247s 4s/step - loss: 0.5143 - recall: 0.7622 - val_loss: 0.5820 - val_recall: 0.7441
Epoch 144/400
62/62 

In [ ]:
plt.style.use('seaborn')
acc = history.history['recall']
val_acc = history.history['val_recall']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Recall')
plt.plot(epochs_range, val_acc, label='Validation Recall')
plt.legend(loc='lower right')
plt.title('Training and Validation Recall')


plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('graphs/simplemodel128x2aug_zoom_bright.png', bbox_inches='tight')
plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory(
        'plants/test',
        target_size=(300, 200),
        batch_size=16,
        shuffle=False,
        class_mode='categorical')

In [ ]:
ben_plants = os.listdir('plants/test/Benign_Plants')
ivy=os.listdir('plants/test/Poison_Ivy/')
oak=os.listdir('plants/test/Poison_Oak')

In [ ]:
y_true=pd.DataFrame()
for i in range(len(ben_plants)):
    num=pd.Series([1,0,0])
    y_true= y_true.append(num, ignore_index=True)
for j in range(len(ivy)):
    num=pd.Series([0,1,0])
    y_true= y_true.append(num, ignore_index=True)
for k in range(len(oak)):
    num=pd.Series([0,0,1])
    y_true= y_true.append(num, ignore_index=True)

In [ ]:
y_hat=(model.predict(test_generator, verbose=1))

In [ ]:
predicted_cat=['Predicted Benign', 'Predicted Poison Ivy', 'Predicted Poison Oak']
real_cat=['Actual Benign', 'Actual Poison Ivy', 'Actual Poison Oak']
sns.heatmap(percent, annot=percent, cmap='YlGn', 
            xticklabels=predicted_cat, 
            yticklabels=real_cat).set_title('Confusion Matrix -Percent Actual Plant Predicted \n built model 128 filters heavy augment with no weights');
plt.savefig('graphs/simplemodel128x2_2aug_zoom_bright.png', bbox_inches='tight')

In [ ]:
def print_model_properties(model, indices = 0):
     for i, layer in enumerate(model.layers[indices:]):
        print("Layer {} | Name: {} | Trainable: {}".format(i+indices, layer.name, layer.trainable))

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_recall",
        # "no longer improving" being defined as "no better than 1e-5 less"
        min_delta=1e-5,
        # "no longer improving" being further defined as "for at least 30 epochs"
        patience=45,
        verbose=1,), 
    keras.callbacks.TensorBoard(run_logdir,
                              histogram_freq=2,
                              write_graph=True,
                              write_images=True),
    keras.callbacks.ModelCheckpoint(filepath='models/'+model_name ,save_best_only=True)]

In [ ]:
optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)

In [ ]:
model.pop()

In [ ]:
model.add(layers.Dropout(0.3)) 
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

In [ ]:
def change_trainable_layers(model, trainable_index):
    for layer in model.layers[:trainable_index]:
        layer.trainable = False
    for layer in model.layers[trainable_index:]:
        layer.trainable = True

In [ ]:
change_trainable_layers(model, 10)